# Project 3 (Ellein Cheng)

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [57]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [58]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [59]:
# frequency table for prestige and whether or not someone was admitted
# Use crosstab - rows are admit, columns are prestige
print pd.crosstab(df['admit'], df['prestige'])

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [60]:
# Use get_dummies - create matrix where columns are the different prestige rankings
# and each row has boolean values where all are zeros except for the column of the 
# correct ranking
dummy_pres = pd.get_dummies(df['prestige'], prefix='pres')
print dummy_pres.head()

   pres_1.0  pres_2.0  pres_3.0  pres_4.0
0         0         0         1         0
1         0         0         1         0
2         1         0         0         0
3         0         0         0         1
4         0         0         0         1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: Four.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [61]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_pres.loc[:, 'pres_1':])
print handCalc.head()

   admit    gre   gpa  pres_1.0  pres_2.0  pres_3.0  pres_4.0
0      0  380.0  3.61         0         0         1         0
1      1  660.0  3.67         0         0         1         0
2      1  800.0  4.00         1         0         0         0
3      1  640.0  3.19         0         0         0         1
4      0  520.0  2.93         0         0         0         1


In [62]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print pd.crosstab(handCalc['admit'], handCalc['pres_1.0'], rownames=['admit'])

pres_1.0    0   1
admit            
0         243  28
1          93  33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [63]:
# Use empirical probabilities with the data for calculations:
# P(success|ranked 1) = 33/total 
# P(failure|ranked 1) = 28/total
# odds = (33/total)/(28/total) = 33/28
odds_ranked1 = 33./28
odds_ranked1

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [64]:
#similar to above
odds_notranked1 = 93./243
odds_notranked1

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [65]:
#odds ratio = odds_ranked1 / odds_notranked1
odds_ratio_R1 = odds_ranked1 / odds_notranked1
odds_ratio_R1

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: Amongst candidates applying to grad schools, odds of being admitted for those who attended a high ranked undergrad school are about 3.07 times higher than those who did not attend a high ranked school.

#### 3.5 Print the cross tab for prestige_4

In [66]:
#crosstab prestige 4 admission 
# frequency table cutting prestige and whether or not someone was admitted
print pd.crosstab(handCalc['admit'], handCalc['pres_4.0'], rownames=['admit'])

pres_4.0    0   1
admit            
0         216  55
1         114  12


#### 3.6 Calculate the OR 

In [67]:
odds_ranked4 = 12./55
print "Odds of admissions (ranked 4):", odds_ranked4

odds_notranked4 = 114./216
print "Odds of admissions (not ranked 4):", odds_notranked4

odds_ratio_R4 = odds_ranked4 / odds_notranked4
print "Odds Ratio (ranked 4):", odds_ratio_R4
offset_R4 = 1 - odds_ratio_R4

print "Or this could be written as 1 - Odds Ratio: ", offset_R4

Odds of admissions (ranked 4): 0.218181818182
Odds of admissions (not ranked 4): 0.527777777778
Odds Ratio (ranked 4): 0.413397129187
Or this could be written as 1 - Odds Ratio:  0.586602870813


#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted is about 58.66% lower for applicants who are from Tier 4 undergrad schools than applicants who did not attend Tier 4 schools.

## Part 4. Analysis

In [69]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_pres.ix[:, 'pres_2':])
print data.head()

   admit    gre   gpa  pres_2.0  pres_3.0  pres_4.0
0      0  380.0  3.61         0         1         0
1      1  660.0  3.67         0         1         0
2      1  800.0  4.00         0         0         0
3      1  640.0  3.19         0         0         1
4      0  520.0  2.93         0         0         1


/Users/echeng1/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [70]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [71]:
#Since the first column is admit, which is the dependent variable,
# we start from the second column onwards (independent variables)
train_cols = data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'pres_2.0', u'pres_3.0', u'pres_4.0', u'intercept'], dtype='object')

#### 4.2 Fit the model

In [72]:
# Model with Logistic Regressions: independent variables - train_cols
# dependent variable - admit
logit = sm.Logit(data['admit'], data[train_cols])
model_result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [73]:
print model_result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 09 Jan 2018   Pseudo R-squ.:                 0.08166
Time:                        20:15:41   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
pres_2.0      -0.6801      0.317     -2.146      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [74]:
print np.exp(model_result.params)

gre          1.002221
gpa          2.180027
pres_2.0     0.506548
pres_3.0     0.262192
pres_4.0     0.211525
intercept    0.020716
dtype: float64


In [75]:
params = model_result.params
conf_interval = model_result.conf_int()
conf_interval['OddsRatio'] = params
conf_interval.columns = ['2.5%', '97.5%', 'OddsRatio']
print np.exp(conf_interval)

               2.5%     97.5%  OddsRatio
gre        1.000074  1.004372   1.002221
gpa        1.136120  4.183113   2.180027
pres_2.0   0.272168  0.942767   0.506548
pres_3.0   0.133377  0.515419   0.262192
pres_4.0   0.093329  0.479411   0.211525
intercept  0.002207  0.194440   0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: Candidates who are from a tier 2 undergrad school have 50% the odds of being admitted to graduate school compared to candidates who are from a tier 1 school.

#### 4.6 Interpret the OR of GPA

Answer: For each unit increase in GPA, the odds of admission into grad school increase by about 118%, assuming other variables are unchanged.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [76]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [77]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [78]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_pres_2 = pd.get_dummies(combos['prestige'], prefix='pres')
dummy_pres_2.columns = ['pres_1.0', 'pres_2.0', 'pres_3.0', 'pres_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_pres_2.ix[:, 'pres_2.0':])
combos.head()

/Users/echeng1/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,gre,gpa,prestige,intercept,pres_2.0,pres_3.0,pres_4.0
0,220.0,2.260000,1.0,1.0,0,0,0
1,220.0,2.260000,2.0,1.0,1,0,0
2,220.0,2.260000,3.0,1.0,0,1,0
3,220.0,2.260000,4.0,1.0,0,0,1
4,220.0,2.453333,1.0,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [80]:
combos['admit_pred'] = model_result.predict(combos[train_cols])

print combos.tail()
len(combos)

       gre       gpa  prestige  intercept  pres_2.0  pres_3.0  pres_4.0  \
395  800.0  3.806667       4.0        1.0         0         0         1   
396  800.0  4.000000       1.0        1.0         0         0         0   
397  800.0  4.000000       2.0        1.0         1         0         0   
398  800.0  4.000000       3.0        1.0         0         1         0   
399  800.0  4.000000       4.0        1.0         0         0         1   

     admit_pred  
395    0.334286  
396    0.734040  
397    0.582995  
398    0.419833  
399    0.368608  


400

#### 5.3 Interpret findings for the last 4 observations

Answer: Given the same GPA 800 and GRE 4.0, applicants who attended a tier 2 undergrad school has a 58% chance of being admitted; applicants who attended a tier 3 undergrad school has 42% chance of being admitted; applicants who attended a tier 4 undergrad school has a 37% chance of being admitted


## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.